In [ ]:
#pip install azure.storage.blob

In [4]:
## Successfully uploads to the database with nested structure

# Retrieve list of blobs from designated container
# Check if blob name is a file name in current directory
# If no same file name in directory, download the file associated with that blobname
# If it downloads the file, then it also opens it, restructures the data into a nested format and imports the data to mongodb

import time
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobServiceClient
import json
from pymongo import MongoClient

# Define function to build nested json 
def make_nested_json(list_of_json):
    participant_name = list_of_json[0]['Participant']
    participant_ID = list_of_json[0]['Participant_ID']
    session_ID = list_of_json[0]['Session_ID']
    streams_and_data = [
        {
            'stream_type': list_of_json[0]['stream_type'], 
            'data': []
        }
    ]
    for i in range(len(list_of_json)):
        streams_in_dict = [a_dict["stream_type"] for a_dict in streams_and_data]
        indexed_stream_type = list_of_json[i]['stream_type']
        if indexed_stream_type in streams_in_dict:                   
            pass
        else:
            streams_and_data.append(
                {
                    'stream_type': indexed_stream_type,
                    'data': []
                }
            )
        streams_in_dict = [a_dict["stream_type"] for a_dict in streams_and_data]
        n = streams_in_dict.index(indexed_stream_type)
        streams_and_data[n]['data'].append(
            {
                'Value': list_of_json[i]['Value'],
                'dateTime': list_of_json[i]['dateTime_Unix']
            }
        )
    nested_json = {
        'Session_ID': session_ID,
        'Participant_ID': participant_ID,
        'Participant': participant_name,
        'Streams': streams_and_data
    } 
    return nested_json  

blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
container = ContainerClient.from_connection_string(conn_str=os.environ.get('AZURE_STORAGE_CONNECTION_STRING'), container_name="container2")

new_files = []
all_files = []

blob_list = container.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    all_files.append(blob.name)
    if blob.name not in os.listdir(os.curdir):
        new_files.append(blob.name)
        t1=time.time()
        blob_client_instance = blob_service_client_instance.get_blob_client("container2", blob.name, snapshot=None)
        with open(blob.name, "wb") as my_blob:
            blob_data = blob_client_instance.download_blob()
            blob_data.readinto(my_blob)
        t2=time.time()
        print(("It takes %s seconds to download "+blob.name + '\n') % (t2 - t1))

### Import new data to mongodb, one file at a time

# Making Connection
myclient = MongoClient("mongodb://localhost:27017/")

# database
db = myclient["Sensorlab"]  #update here to switch database

# Create or switch to collection
Collection = db["test_nestedJSON"]  #update here to switch collection

for i in range(len(new_files)):
    with open(new_files[i]) as f:
        content = f.readlines()
    file_data = []
    for data in content:
        file_data.append(json.loads(data))

    nested_json_file_data = make_nested_json(file_data)

    if isinstance(nested_json_file_data, list):
        Collection.insert_many(nested_json_file_data)
    else:
        Collection.insert_one(nested_json_file_data)

    print(f"Imported {new_files[i]} to the database.")



0_18fc7328361948c0865df2f0c53aa008_1.json

0_2f7120ed65194093a6df6dacd979d706_1.json

0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json

It takes 2.759887933731079 seconds to download 0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json

0_4862fd0f886945c6b5fa1ed9b391e807_1.json

0_5e20c79f9efc42fc9c09b4443a6646de_1.json

0_603c31b59afd4c0eb25fade694cdb24c_1.json

0_6bf71ccc6254498bb6e52e10f45490d8_1.json

0_743237a427664f57b35d3bd642572e6d_1.json

0_75d343691ca849bdb190d98b11f7edc9_1.json

0_8223e703445d414c8f34b45d71a9fe35_1.json

0_8d054dcd0a7b487585f77bfee87ff543_1.json

0_a48ce3b7dec3442f963db431ff2e4da5_1.json

0_bc12f2ee7a2e41f9b463001439e8ae94_1.json

0_bcf7c48af48e4ff0bde2b79cf6606216_1.json

0_ddb730a7073646d995d0534ada10ceac_1.json

It takes 2.5093915462493896 seconds to download 0_ddb730a7073646d995d0534ada10ceac_1.json

0_f1ba9f5911b743daae5c728be89b5fb0_1.json

0_f1e1947720fb4072a8b325428be98a10_1.json

Imported 0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json to the database.
Imported 0_ddb730a70

In [18]:
## Uploads to db without nested structure

# Retrieve list of blobs from designated container
# Check if blob name is a file name in current directory
# If no same file name in directory, download the file associated with that blobname
# If it downloads the file, then it also opens it and imports the data to mongodb

import time
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobServiceClient
import json
from pymongo import MongoClient

blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
container = ContainerClient.from_connection_string(conn_str=os.environ.get('AZURE_STORAGE_CONNECTION_STRING'), container_name="container2")

new_files = []
all_files = []

blob_list = container.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    all_files.append(blob.name)
    if blob.name not in os.listdir(os.curdir):
        new_files.append(blob.name)
        t1=time.time()
        blob_client_instance = blob_service_client_instance.get_blob_client("container2", blob.name, snapshot=None)
        with open(blob.name, "wb") as my_blob:
            blob_data = blob_client_instance.download_blob()
            blob_data.readinto(my_blob)
        t2=time.time()
        print(("It takes %s seconds to download "+blob.name + '\n') % (t2 - t1))

### Import new data to mongodb, one file at a time

# Making Connection
myclient = MongoClient("mongodb://localhost:27017/")

# database
db = myclient["Sensorlab"]  #update here to switch database

# Create or switch to collection
Collection = db["E4_data"]  #update here to switch collection

for i in range(len(new_files)):
    with open(new_files[i]) as f:
        content = f.readlines()
    file_data = []
    for data in content:
        file_data.append(json.loads(data))

    if isinstance(file_data, list):
        Collection.insert_many(file_data)
    else:
        Collection.insert_one(file_data)
    print(f"Imported {new_files[i]} to the database.")



0_18fc7328361948c0865df2f0c53aa008_1.json

0_2f7120ed65194093a6df6dacd979d706_1.json

0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json

It takes 3.7836661338806152 seconds to download 0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json

0_4862fd0f886945c6b5fa1ed9b391e807_1.json

0_5e20c79f9efc42fc9c09b4443a6646de_1.json

0_603c31b59afd4c0eb25fade694cdb24c_1.json

0_6bf71ccc6254498bb6e52e10f45490d8_1.json

0_743237a427664f57b35d3bd642572e6d_1.json

0_75d343691ca849bdb190d98b11f7edc9_1.json

0_8223e703445d414c8f34b45d71a9fe35_1.json

0_8d054dcd0a7b487585f77bfee87ff543_1.json

0_a48ce3b7dec3442f963db431ff2e4da5_1.json

0_bcf7c48af48e4ff0bde2b79cf6606216_1.json

0_ddb730a7073646d995d0534ada10ceac_1.json

It takes 3.704059600830078 seconds to download 0_ddb730a7073646d995d0534ada10ceac_1.json

0_f1e1947720fb4072a8b325428be98a10_1.json

Imported 0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json to the database.
Imported 0_ddb730a7073646d995d0534ada10ceac_1.json to the database.


In [1]:
import os
os.environ['AZURE_STORAGE_CONNECTION_STRING'] = "DefaultEndpointsProtocol=https;AccountName=uaholodecksensorlab;AccountKey=GX+Fn1hVo3RDWRGuCxMAVDVFA/maCM2NdGx4Kffv4tWnG6DU8C1NOVH5Rv694e3HVNCmlinkeAKMgnXBvsr7nA==;EndpointSuffix=core.windows.net"

STORAGEACCOUNTURL= "https://uaholodecksensorlab.blob.core.windows.net"
STORAGEACCOUNTKEY= "GX+Fn1hVo3RDWRGuCxMAVDVFA/maCM2NdGx4Kffv4tWnG6DU8C1NOVH5Rv694e3HVNCmlinkeAKMgnXBvsr7nA=="

In [5]:
nested_json_file_data

{'Session_ID': '0011628615126.8852172',
 'Participant_ID': '001',
 'Participant': 'Tony Banks',
 'Streams': [{'stream_type': 'E4_Bvp',
   'data': [{'Value': -87.41513, 'dateTime': '1628615128.94081'},
    {'Value': -86.01074, 'dateTime': '1628615128.95644'},
    {'Value': -82.96423, 'dateTime': '1628615128.97206'},
    {'Value': -78.10001, 'dateTime': '1628615128.98769'},
    {'Value': -72.33652, 'dateTime': '1628615129.00331'},
    {'Value': -67.32547, 'dateTime': '1628615129.01894'},
    {'Value': -64.64914, 'dateTime': '1628615129.03457'},
    {'Value': -65.0592, 'dateTime': '1628615129.05019'},
    {'Value': -68.12302, 'dateTime': '1628615129.06582'},
    {'Value': -72.42111, 'dateTime': '1628615129.08144'},
    {'Value': -76.20532, 'dateTime': '1628615129.09707'},
    {'Value': -78.21729, 'dateTime': '1628615129.11269'},
    {'Value': -78.24643, 'dateTime': '1628615129.12832'},
    {'Value': -77.18845, 'dateTime': '1628615129.14395'},
    {'Value': -76.5477, 'dateTime': '162861512